# Подбор гипперпараметров нейросети при помощи Keras Tuner Hyperband

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
from keras_tuner import RandomSearch, BayesianOptimization, Hyperband, Objective
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import regularizers

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from python_files.data import data
data_ = data.copy()

In [3]:
from python_files.matrix.mat_split_data import X_train_mat, X_test_mat, y_train_mat, y_test_mat
from python_files.matrix.mat_col_list import mat_x_col_list, mat_col_list_norm, mat_col_list_std

### Preprocessing

In [4]:
# нормализуем и стандартизируем X_train, X_test
X_train_mat_prep = PowerTransformer().fit_transform(X_train_mat) 
X_test_mat_prep = PowerTransformer().fit_transform(X_test_mat) 

In [5]:
# нормализуем и стандартизируем y_train, y_test
y_train_mat_df = pd.DataFrame(y_train_mat, columns = [data_.columns[0]])
y_test_mat_df = pd.DataFrame(y_test_mat, columns = [data_.columns[0]])

pt_y_train = PowerTransformer()
pt_y_test = PowerTransformer()

y_train_mat_prep = pt_y_train.fit_transform(y_train_mat_df) 
y_test_mat_prep = pt_y_test.fit_transform(y_test_mat_df) 

### Model - MLP

In [6]:
# определяем пользовательскую функцию - нейросеть 
def MLP_model(hp):
    model = Sequential()
    model.add(Dense(units = hp.Float('units_input_1', min_value = 8, max_value = 56, step = 4),
                            activation = hp.Choice('activation_1', values = ['relu','tanh','sigmoid','elu','selu']),
                            input_shape = (12,), 
                            bias_initializer = hp.Choice('bs_init_1', values = ['zeros', 'he_normal', 'glorot_normal',
                                                                               'he_uniform', 'glorot_uniform']),
                            kernel_initializer = hp.Choice('kn_init_1', values = ['he_normal', 'glorot_normal',
                                                                                 'he_uniform', 'glorot_uniform']),
                            activity_regularizer='l1_l2'))
    
    model.add(Dropout(rate = hp.Float('dropout_1', min_value = 0.0, max_value = 0.5, step = 0.1)))
    
    model.add(Dense(units = hp.Float('units_input_2', min_value = 8, max_value = 56, step = 4),
                            activation = hp.Choice('activation_2', values = ['relu', 'tanh', 'sigmoid', 'elu', 'selu']),
                            bias_initializer = hp.Choice('bs_init_2', values = ['zeros', 'he_normal', 'glorot_normal',
                                                                               'he_uniform', 'glorot_uniform']),
                            kernel_initializer = hp.Choice('kn_init_2', values = ['he_normal', 'glorot_normal',
                                                                                 'he_uniform', 'glorot_uniform']),
                            activity_regularizer='l1_l2'))
    
    model.add(Dropout(rate = hp.Float('dropout_2', min_value = 0.0, max_value = 0.5, step = 0.1)))
    
    model.add(Dense(units = hp.Float('units_input_3', min_value = 8, max_value = 56, step = 4),
                            activation = hp.Choice('activation_3', values = ['relu', 'tanh', 'sigmoid', 'elu', 'selu']),
                            bias_initializer = hp.Choice('bs_init_3', values = ['zeros', 'he_normal', 'glorot_normal',
                                                                               'he_uniform', 'glorot_uniform']),
                            kernel_initializer = hp.Choice('kn_init_3', values = ['he_normal', 'glorot_normal',
                                                                                 'he_uniform', 'glorot_uniform']),
                            activity_regularizer='l1_l2'))
    
    model.add(Dropout(rate = hp.Float('dropout_3', min_value = 0.1, max_value = 0.5, step = 0.1)))
    
    model.add(Dense(1, activation = 'linear', 
                    bias_initializer = hp.Choice('bs_init_ex', values = ['zeros', 'he_normal', 'he_uniform']),
                    kernel_initializer = hp.Choice('kn_init_ex', values = ['he_normal', 'he_uniform']),
                    activity_regularizer='l1_l2'))
   
    sgd = SGD(momentum=0.9, nesterov=True)
    model.compile(loss = 'mean_absolute_error', optimizer = hp.Choice('optimizer', values = ['sgd','Adam','Adadelta','RMSprop']),
                  metrics=['mean_absolute_error'])
    return model

### Keras Tuner

In [7]:
# определяем кeras_tuner
tuner = Hyperband(
                     MLP_model,
                     objective = Objective("val_mean_absolute_error", direction="min"),
                     max_epochs = 300,
                     hyperband_iterations = 3,
                     seed = 7,
                     directory = '/keras_tuner/hyperband_2',
                     #overwrite = True
                    )

ear_stop = EarlyStopping(monitor = 'val_mean_absolute_error', patience = 30)

reduce_lr = ReduceLROnPlateau(monitor = 'val_mean_absolute_error', factor = 0.2,
                              patience = 5, min_lr = 0.0000001, min_delta = 0.001)

In [8]:
# пространство подбираемых гипперпараметров
tuner.search_space_summary()

Search space summary
Default search space size: 18
units_input_1 (Float)
{'default': 8.0, 'conditions': [], 'min_value': 8.0, 'max_value': 56.0, 'step': 4, 'sampling': 'linear'}
activation_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu', 'selu'], 'ordered': False}
bs_init_1 (Choice)
{'default': 'zeros', 'conditions': [], 'values': ['zeros', 'he_normal', 'glorot_normal', 'he_uniform', 'glorot_uniform'], 'ordered': False}
kn_init_1 (Choice)
{'default': 'he_normal', 'conditions': [], 'values': ['he_normal', 'glorot_normal', 'he_uniform', 'glorot_uniform'], 'ordered': False}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units_input_2 (Float)
{'default': 8.0, 'conditions': [], 'min_value': 8.0, 'max_value': 56.0, 'step': 4, 'sampling': 'linear'}
activation_2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu', 'selu'], 'ordered': F

In [9]:
# подбираем гипперпараметры
tuner.search(X_train_mat_prep, y_train_mat_prep,
             #epochs = 200,
             validation_split = 0.3,
             callbacks = [ear_stop, reduce_lr],
             verbose = 1,
             #directory = '/keras_tuner/untitled_project'             
             )

Trial 2175 Complete [00h 00m 08s]
val_mean_absolute_error: 0.7909500598907471

Best val_mean_absolute_error So Far: 0.7752560377120972
Total elapsed time: 03h 41m 48s
INFO:tensorflow:Oracle triggered exit


### Результаты работы Keras Tuner

In [10]:
# отчет по результатам каждой попытки keras_tuner
tuner.results_summary()

Results summary
Results in /keras_tuner/hyperband_2\untitled_project
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 1263 summary
Hyperparameters:
units_input_1: 56.0
activation_1: selu
bs_init_1: glorot_normal
kn_init_1: glorot_normal
dropout_1: 0.4
units_input_2: 44.0
activation_2: relu
bs_init_2: he_uniform
kn_init_2: he_uniform
dropout_2: 0.0
units_input_3: 28.0
activation_3: sigmoid
bs_init_3: glorot_normal
kn_init_3: he_normal
dropout_3: 0.4
bs_init_ex: he_uniform
kn_init_ex: he_normal
optimizer: RMSprop
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.7752560377120972

Trial 1010 summary
Hyperparameters:
units_input_1: 56.0
activation_1: elu
bs_init_1: glorot_uniform
kn_init_1: he_uniform
dropout_1: 0.4
units_input_2: 24.0
activation_2: tanh
bs_init_2: zeros
kn_init_2: he_normal
dropout_2: 0.1
units_input_3: 12.0
activation_3: relu
bs_init_3: zeros
kn_init_3: he_normal
dropout_3: 0.1
bs_init_ex: zeros
kn_ini

In [11]:
best_hp = tuner.get_best_hyperparameters()[0]

In [12]:
best_hp.values

{'units_input_1': 56.0,
 'activation_1': 'selu',
 'bs_init_1': 'glorot_normal',
 'kn_init_1': 'glorot_normal',
 'dropout_1': 0.4,
 'units_input_2': 44.0,
 'activation_2': 'relu',
 'bs_init_2': 'he_uniform',
 'kn_init_2': 'he_uniform',
 'dropout_2': 0.0,
 'units_input_3': 28.0,
 'activation_3': 'sigmoid',
 'bs_init_3': 'glorot_normal',
 'kn_init_3': 'he_normal',
 'dropout_3': 0.4,
 'bs_init_ex': 'he_uniform',
 'kn_init_ex': 'he_normal',
 'optimizer': 'RMSprop',
 'tuner/epochs': 4,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 4,
 'tuner/round': 0}

In [13]:
best_model = tuner.get_best_models()[0]

In [14]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                728       
                                                                 
 dropout (Dropout)           (None, 56)                0         
                                                                 
 dense_1 (Dense)             (None, 44)                2508      
                                                                 
 dropout_1 (Dropout)         (None, 44)                0         
                                                                 
 dense_2 (Dense)             (None, 28)                1260      
                                                                 
 dropout_2 (Dropout)         (None, 28)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 2

### Prediction

In [15]:
preds = best_model.predict(X_test_mat_prep)

10/10 [==============================] - 3s 5ms/step


In [16]:
preds_train = best_model.predict(X_train_mat_prep)

23/23 [==============================] - 0s 2ms/step


### Evaluation

In [17]:
r2_score(y_train_mat_prep, preds_train), r2_score(y_test_mat_prep, preds)

(0.01802809075490941, -0.03485797742660779)

In [18]:
preds_inv = pt_y_test.inverse_transform(preds)

C:\anaconda3\envs\VKR\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


In [19]:
mean_absolute_error(y_test_mat, preds_inv)

0.6939731460311989

In [20]:
mean_squared_error(y_test_mat, preds_inv)

0.7821167085574426

In [21]:
y_test_mat.max() - y_test_mat.min() 

5.202338993519127

In [22]:
mean_absolute_error(y_test_mat, preds_inv) * 100 / (y_test_mat.max() - y_test_mat.min())

13.339637168891219